In [28]:
import numpy as np
from scipy.integrate import odeint
from bokeh.plotting import figure, output_notebook, show

# specify number of steps
ns = 1200
# define time points
t = np.linspace(0,ns,ns+1)

# mode (manual=0, automatic = 1)
mode = 1

class model(object):
    # process model
    Kp = 2.0
    taup = 200.0
    thetap = 0.0

class pid(object):
    # PID tuning
    Kc = 2.0
    tauI = 10.0
    tauD = 0.0
    sp = []

# Define set point
sp = np.zeros(ns+1)  # set point
sp[50:600] = 10.0
sp[600:] = 0
pid.sp = sp

def process(y,t,u,Kp,taup):
    # Kp =  process gain
    # taup = prcoess time constant
    dydt = -y/taup + Kp/taup * u
    return dydt

def calc_response(t,mode,xm,xc):
    # t = time points
    # mode (manual = 0,automatic = 1)
    # process model
    Kp = xm.Kp
    taup = xm.taup
    thetap = xm.thetap
    # specify number of steps
    ns = len(t)-1
    # PID tuning
    Kc = xc.Kc
    tauI = xc.tauI
    tauD = xc.tauD
    sp = xc.sp  # set point
    
    delta_t = t[1]-t[0]
    
    # storage for recording values
    op = np.zeros(ns+1)  # controller output
    pv = np.zeros(ns+1)  # process variable
    e = np.zeros(ns+1)   # error
    ie = np.zeros(ns+1)  # integral of the error
    dpv = np.zeros(ns+1) # derivative of the pv
    P = np.zeros(ns+1)   # proportional
    I = np.zeros(ns+1)   # integral
    D = np.zeros(ns+1)   # derivative
    
    # step input for manual control
    if mode == 0:
        op[100:] = 2
    
    # upper and lower limits on OP
    op_hi = 100.0
    op_low = 0.0
    
    # simulate time delay
    ndelay = int(np.ceil(thetap/delta_t))
    
    # loop through time steps
    for i in range(0,ns):
        e[i] = sp[i]-pv[i]
        if i>=1:  # calculate starting on second cycle
            dpv[i] = (pv[i]-pv[i-1])/delta_t
            ie[i] = ie[i-1]+e[i]*delta_t
        P[i] = Kc*e[i]
        I[i] = Kc/tauI * ie[i]   # p only
        D[i] = -Kc*tauD*dpv[i]
        if mode == 1:
            op[i] = op[0] + P[i] + I[i] + D[i]
        if op[i]>op_hi:   # check upper limit
            op[i] = op_hi
            ie[i] = ie[i] - e[i]*delta_t # anti-reset windup
        if op[i]<op_low:  # check lower limit
            op[i] = op_low
            ie[i] = ie[i] - e[i] * delta_t  # anti-reset windup
        # implement time delay
        iop = max(0,i-ndelay)
        y = odeint(process,pv[i],[0,delta_t],args=(op[iop],Kp,taup))
        pv[i+1]=y[-1]
    op[ns]=op[ns-1]
    ie[ns] = ie[ns-1]
    P[ns] = P[ns-1]
    I[ns] = I[ns-1]
    D[ns] = D[ns-1]
    return (pv,op)

def plot_response(mode,t,pv,op,sp):
    # plot results
    output_notebook()
    p1 = figure(plot_width=400, plot_height=200,y_axis_label='Process Output',x_axis_label='Time')
    if (mode==1):
        p1.line(t,sp,line_color = 'red',line_width = 3,legend='Set Point (SP)')
    p1.line(t,pv,line_color = 'blue',line_width = 3, line_dash = '4 4', legend='Process Variable (PV)')
    
    
    p2 = figure(plot_width=400, plot_height=200,y_axis_label='Process Input',x_axis_label = 'Time')
    p2.line(t,op,line_color = 'red',line_width = 3,legend='Controller Output(OP)')
    show(p1)
    show(p2)


# calculate step response
model.Kp = 2.0
model.taup = 200.0
model.thetap = 10.0
mode = 0
(pv,op) = calc_response(t,mode,model,pid)
plot_response(mode,t,pv,op,sp)            

Loading BokehJS ...

In [29]:
# PI control
pid.Kc = 2.0
pid.tauI = 10.0
pid.tauD = 0.0
mode = 1
(pv,op) = calc_response(t,mode,model,pid)
plot_response(mode,t,pv,op,sp)


Loading BokehJS ...

In [30]:
# PI control
pid.Kc = 2.0
pid.tauI = 100.0
pid.tauD = 0.0
mode = 1
(pv,op) = calc_response(t,mode,model,pid)
plot_response(mode,t,pv,op,sp)


Loading BokehJS ...